## Lab 4 - Samuel Akinnusi

#### Question 1
Create a point feature class (shapefile – q1.shp) from the locations.txt file. (4 Points)

In [11]:
import arcpy

#create the workspace environment and set the output result to always overwrite 
arcpy.env.workspace = 'T:\\Students\\sakinnus\\Submissions\\Lab4_Submission\\'
arcpy.env.overwriteOutput = True

#Locate the text file and read all the lines except the header line
locationFile = open('locations.txt', 'r')
textLines = locationFile.readlines()[1:]

#create new shapefile 'points.shp' and add the field names
arcpy.CreateFeatureclass_management(arcpy.env.workspace, 'points1.shp', 'point')
arcpy.AddField_management('points1.shp', 'LocID', 'Text', '','',10)

#Use the insertCursor module function to insert new rows into the shapefile
locationCursor = arcpy.da.InsertCursor('points1.shp',['LocID', 'SHAPE@XY']) #set the template of the shapefile to insert values

for aLine in textLines:
    aListValues = aLine.split(',') #split the columns in the text files with comma ','
    locationID = aListValues[0]
    x_cord = float(aListValues[1])
    y_cord = float(aListValues[2])
    
    locationCursor.insertRow([locationID,(x_cord,y_cord)]) #insert the new row for all rows in the text file
    
del locationCursor
locationFile.close()

#### Question 2	
In q2.py, calculate the mean center for q1.shp and add the point to q1.shp, save the new shapefile as q2.shp. (4 Points)

In [ ]:
#create the workspace environment and set the output result to always overwrite 
arcpy.env.workspace = 'T:\\Students\\sakinnus\\Submissions\\Lab4_Submission\\'
arcpy.env.overwriteOutput = True

#Locate the text file and read all the lines except the header line
locationFile = open('locations.txt', 'r')
textLines = locationFile.readlines()[1:]

arcpy.CreateFeatureclass_management(arcpy.env.workspace, 'q2.shp', 'point')
arcpy.AddField_management('q2.shp', 'LocID', 'Text', '','',10)

locationCursor = arcpy.da.InsertCursor('q2.shp',['LocID', 'SHAPE@XY'])

x=0
y=0
count =0
for aLine in textLines:
    aListValues = aLine.split(',')
    locationID = aListValues[0]
    x_cord = float(aListValues[1])
    y_cord = float(aListValues[2])
    x=x_cord + x
    y=x_cord + y
    count = count +1

    locationCursor.insertRow([locationID,(x_cord,y_cord)])

x_mc = x/count
y_mc = y/count
locationCursor.insertRow(['mc',(x_mc,y_mc)])

del locationCursor
locationFile.close()

#### Question 3
Write a script (q3.py) to convert oneline.txt into polyline feature class (shapefile). Oneline.txt includes multiple vertices for one polyline object. Each row represents one vertex – 
177030005600   904     2.5     206   904         29.409337        -92.888029………...
* The first column represents the polyline’s ID, and last two columns represent a vertex’s latitude and longitude. (4 Points)

In [5]:
import arcpy

#create the workspace environment and set the output result to always overwrite 
arcpy.env.workspace = 'T:\\Students\\sakinnus\\Submissions\\Lab4_Submission\\'
arcpy.env.overwriteOutput = True

#Locate the text file and read all the lines except the header line
onelineFile = open('oneline.txt', 'r')
textLines = onelineFile.readlines()

#create a new polyline shapefile and add a new field "polylineID"
arcpy.CreateFeatureclass_management(arcpy.env.workspace, 'PolylineQ3.shp', 'polyline')
arcpy.AddField_management('PolylineQ3.shp', 'polylineID', 'Text', '','',10)

#call the InsertCursor tool and set the template to allow addition of new record into the new shapefile 
polylineCursor = arcpy.da.InsertCursor('PolylineQ3.shp',['polylineID', 'SHAPE@'])

#create an empty array
points = arcpy.Point()
array = arcpy.Array()

#loop through each line in the text file and get the x and y cordinates and the lineIDs, then add them to the empty array 
for aLine in textLines:
    aListValues = aLine.split()
    lineID = aListValues[0]
    x_cord = float(aListValues[6])
    y_cord = float(aListValues[5])
    points.X = x_cord
    points.Y = y_cord
    array.add(points)

#create the polyline with the set of array points using the Arcpy polyline class
polyline = arcpy.Polyline(array)

#insert the new polyline record as a row in the polyline attribute table
polylineCursor.insertRow([lineID,polyline])

del polylineCursor
onelineFile.close()

#### Question 4
Write a script (q4.py) to convert test.txt into a polyline feature class and save the result to your Lab6 folder. Test.txt contains more than one polyline object. Each polyline object has a unique line ID, such as 177030005600, 177030009200, etc. Please use the Line ID to identify each line segment. You can save the result as a shapefile. Once succeeded, apply your code to allLines.txt, which has thousands of records – more than 20 Mbs. (4 Points)

In [35]:
#create the workspace environment and set the output result to always overwrite 
arcpy.env.workspace = 'T:\\Students\\sakinnus\\Submissions\\Lab4_Submission\\'
arcpy.env.overwriteOutput = True

#Locate the text file and read all the lines except the header line
allLines = open('allLines.txt', 'r')
textLines = testFile.readlines()

#create a new polyline shapefile and add a new field "polylineID"
arcpy.CreateFeatureclass_management(arcpy.env.workspace, 'PolylineQ4.shp', 'polyline')
arcpy.AddField_management('PolylineQ4.shp', 'polylineID', 'Text', '','',10)

#call the InsertCursor tool and set the template to allow addition of new record into the new shapefile 
polylineCursor = arcpy.da.InsertCursor('PolylineQ4.shp',['polylineID', 'SHAPE@'])

array = [(1,2),(2,3)]
#create an empty array
points = arcpy.Point()
array = arcpy.Array()

#loop through each line in the text file and get the x and y cordinates and the lineIDs, then add them to the empty array
previousID = textLines[0].split()[0]
for aLine in textLines:
    aListValues = aLine.split()
    lineID = aListValues[0]
    x_cord = float(aListValues[6])
    y_cord = float(aListValues[5])
    points.X = x_cord
    points.Y = y_cord
    array.add(points)
    if lineID != previousID:
        polyline = arcpy.Polyline(array)
        polylineCursor.insertRow([lineID, polyline])
        previousID = lineID
        array.removeAll()

del polylineCursor
allLines.close()

#### Question 5
write a script (q5.py) to perform the following procedures: (4 Points)
* a.	List all numerical fields from states2010.shp for a user to choose. For example, <1>POP2000		<2>POP2010		<3>POP00_SQMI …, and the user can choose POP2000 by typing 1 and hit “enter” key.
* b.	Once your script receives the input from the user, compute (1) total value of the selected field (2) maximum value of the chosen field (3) average value of the chosen field
* c.	Prompt the results to the user.
* d.	If there is no “popchg” field, create a “popchg” field for states2010.shp, then calculate population change percentage from 2000 to 2010 for each state and populate this field. To add a field to a feature class, you can use AddField_management function. For examples of using this function, you can search ArcGIS10 desktop help.


In [12]:
import arcpy

#create the workspace environment and set the output result to always overwrite 
arcpy.env.workspace = 'C:\\Users\\Samuel\\Documents\\GEOG_592\\Lab4\\'
arcpy.env.overwriteOutput = True

#call the shapefile and get all the numeric fields or columns
ShapeFile = "states2010.shp"
numericalFields = arcpy.ListFields(ShapeFile, "", 'Double') + arcpy.ListFields(ShapeFile, "", 'Integer') + arcpy.ListFields(ShapeFile, "", 'Float')
count = 0

#get all the field names in the nuemrical table and print them
for afield in numericalFields:
    print("<%d>" % count + afield.name)
    count+=1
#get user's input to select the specific column using the index value    
userinput = int(input("Please choose a field by typing the corresponding index number.\n"))
statCursor = arcpy.da.SearchCursor(ShapeFile, [numericalFields[userinput].name])

total = 0
maximum = -9999999999
count = 0

#loop through the specified column/field and calculate the statistics
for aRow in statCursor:
    total=int(aRow[0])+total #Calculate total and maximum in the loop
    count = count+1
    #print(aRow)
    #Calculate the maximum value & mean
    if aRow[0] > maximum:
        maximum = aRow[0]
mean = total/count

#Print the total, mean and maximum values
print("The total is {}".format(total))
print("The mean is {}".format(mean))
print("The maximum value is {}".format(maximum))

#del statCursor

<0>POP00_SQMI
<1>POP10_SQMI
<2>MED_AGE
<3>MED_AGE_M
<4>MED_AGE_F
<5>AVE_HH_SZ
<6>AVE_FAM_SZ
<7>NO_FARMS07
<8>AVG_SIZE07
<9>CROP_ACR07
<10>AVG_SALE07
<11>popchg
<12>POP2000
<13>POP2010
<14>WHITE
<15>BLACK
<16>AMERI_ES
<17>ASIAN
<18>HAWN_PI
<19>OTHER
<20>MULT_RACE
<21>HISPANIC
<22>MALES
<23>FEMALES
<24>AGE_UNDER5
<25>AGE_5_17
<26>AGE_18_21
<27>AGE_22_29
<28>AGE_30_39
<29>AGE_40_49
<30>AGE_50_64
<31>AGE_65_UP
<32>HOUSEHOLDS
<33>HSEHLD_1_M
<34>HSEHLD_1_F
<35>MARHH_CHD
<36>MARHH_NO_C
<37>MHH_CHILD
<38>FHH_CHILD
<39>FAMILIES
<40>HSE_UNITS
<41>VACANT
<42>OWNER_OCC
<43>RENTER_OCC
<44>SQMI
<45>OID_1
<46>R2010
<47>R2010Dens
<48>FID
<49>Shape
<50>STATE_NAME
<51>STATE_FIPS
<52>SUB_REGION
<53>STATE_ABBR
<54>POP2000
<55>POP2010
<56>POP00_SQMI
<57>POP10_SQMI
<58>WHITE
<59>BLACK
<60>AMERI_ES
<61>ASIAN
<62>HAWN_PI
<63>OTHER
<64>MULT_RACE
<65>HISPANIC
<66>MALES
<67>FEMALES
<68>AGE_UNDER5
<69>AGE_5_17
<70>AGE_18_21
<71>AGE_22_29
<72>AGE_30_39
<73>AGE_40_49
<74>AGE_50_64
<75>AGE_65_UP
<76>MED_AGE
<77>MED_

In [33]:
#Search for popchg field, if found, do nothing, otherwise, create popchg field, and populate the field
def findField(fieldList, fieldName):
    for fld in fieldList:
        if fld.name == fieldName:
            return True
    return False
# Test to see if the "popchg" field exists
if findField(numericalFields, "popchg"):
    print ("popchg field already exists, deleting .. ")
    arcpy.DeleteField_management(ShapeFile, ["popchg"])

# Add a float field named "popchg" here
arcpy.AddField_management(ShapeFile, "popchg", "FLOAT","10","3")
rows = arcpy.da.UpdateCursor(ShapeFile, ["POP2000","POP2010","popchg"])
for row in rows:
    # Get pop2000 and pop2010 values, calculate the pop change value (row[2] = ...)
    row[2]=((row[1]-row[0])/row[0])*100
    rows.updateRow(row)
del row, rows

popchg field already exists, deleting .. 
